## Import Required Files

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
## change it to the unzip path of the downloaded dataset..
data_folder = r'/Users/anurag/Downloads/movie genre classification/Scripts'

In [4]:
all_files = os.listdir(data_folder)
print('Total Number of Files :', len(all_files))

Total Number of Files : 2827


# Read Train Files

In [24]:
train_df = pd.read_csv('Train.csv')
train_df.head()

,File_Name,Labels
0,file_2180.txt,8
1,file_693.txt,4
2,file_2469.txt,6
3,file_2542.txt,6
4,file_378.txt,16


In [28]:
test_df = pd.read_csv('Test.csv', names=['File_Name'])
# test_df.columns = ['File_Name']
test_df

,File_Name
0,file_2300.txt
1,file_809.txt
2,file_1383.txt
3,file_983.txt
4,file_1713.txt
...,...
844,file_2474.txt
845,file_863.txt
846,file_1547.txt
847,file_1292.txt


In [7]:
## let's read the text scripts in the train and test dataframes..

train_df['Script'] = [open(data_folder + os.sep + file, "r").read() for file in train_df['File_Name']]
test_df['Script'] = [open(data_folder + os.sep + file, "r").read() for file in test_df['File_Name']]

# Lets look at a script file after Reading..

In [8]:
#lets check one of the scripts..
print(train_df['Script'][4][:3000])

<b><!--

</b>if (window!= top)

top.location.href=location.href

<b>// -->

</b>

The Abyss - by James Cameron 

                                  THE ABYSS

                            AN ORIGINAL SCREENPLAY

                                      BY

                                JAMES CAMERON

                               August 2, 1988

                             Director's Revision

------------------------------------------------------------------------------

                                  THE ABYSS

OMITTED                                                                 1

OMITTED                                                                 2

TITLE: THE ABYSS -- ON BLACK, DISSOLVING TO COBALT BLUE

EXT. OCEAN/UNDERWATER -- DAY                                            3

Blue, deep and featureless, the twilight of five hundred feet down.

PROPELLER SOUND.  Materializing out of the blue limbo is the enormous but

sleek form of an Ohio-class SSBN ballistic missile su

In [9]:
print(test_df['Script'][4][:3000])

                                          KING KONG

                                          Written by

                        Fran Walsh, Philippa Boyens and Peter Jackson

                                     Based on a Story by

                             Merian C. Cooper and Edgar Wallace

                                                                   1.

          EXT. CENTRAL PARK - DAY

          CLOSE ON: A scrawny MONKEY scratches.

          ANGLES ON: Defeated, listless ANIMALS, in the bleak environs of a

          dilapidated ZOO.

          WIDER: It is CENTRAL PARK ZOO in depression era NEW YORK. The PARK

          itself is like a GARBAGE DUMP, dotted with squalid SHANTY TOWNS.

          Against these BLEAK IMAGES, the SOUND of a BRIGHT, BRASSY SONG

          fades up: Al Jolson, singing "I'm Sitting on Top of the World".

          The sky line of MANHATTAN rises in the background, a grim steaming

          jungle on this cold FALL day.

                 

In [13]:
train_df.Labels.value_counts()

6     405
19    261
4     243
0     203
5     141
15    134
1     116
16    109
11    104
8      79
14     75
7      27
2      25
20     18
13     15
21      9
12      4
9       3
3       2
17      2
10      2
18      1
Name: Labels, dtype: int64

In [14]:
train_df = train_df.append(train_df[train_df['Labels'] == 18].reset_index(drop=True))

In [12]:
# train_df = train_df.drop(1859, axis=0)
# train_df.shape

In [103]:
# test_df = test_df.drop(407, axis=0)
# test_df.shape

In [12]:
# !pip install keras
# !pip install nltk

# Import the Modeling Libraries 

In [15]:
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords


Using TensorFlow backend.
/Users/anurag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/anurag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/anurag/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/anurag/opt/anaconda3/lib/python3.7

Let's load the datasets

In [16]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anurag/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Define the Scoring Metric

In [17]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    
    """Multi class version of Logarithmic Loss metric.
    
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    print(clip)
    vsota = np.sum(actual * np.log(clip.tolist()))
    return -1.0 / rows * vsota

### Before going further it is important that we split the data into training and validation sets. We can do it using 
#### `train_test_split` from the `model_selection` module of scikit-learn.

In [21]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train_df.Script.values, train_df.Labels, 
                                                  stratify=train_df.Labels, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [22]:
print (xtrain.shape)
print (xvalid.shape)

(1781,)
(198,)


# Building Basic Models

### Let's start building our very first model. 

### Our very first model is a simple TF-IDF (Term Frequency - Inverse Document Frequency) followed by a simple Logistic Regression.

In [23]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))

xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

## TFIDF on test data

In [115]:
x_test_tfv = tfv.transform(test_df['Script'].values)

In [116]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

/Users/anurag/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/anurag/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


logloss: 2.486 


# Submission 

### Let's predict on the entire test data...

In [117]:
test_set_preds = pd.DataFrame(columns = train_df.Genre.unique().tolist())
test_set_preds.insert(0, 'File_name', test_df.File_Name)

In [118]:
test_set_preds[train_df.Genre.unique().tolist()] = clf.predict_proba(x_test_tfv)

In [28]:
test_set_preds.shape

NameError: name 'test_set_preds' is not defined

In [120]:
test_set_preds.head()

,File_name,Fantasy,Comedy,Drama,Sci-Fi,Romance,Thriller,Adventure,Mystery,Action,...,Family,Biography,Musical,War,Western,Music,History,Short,Film-Noir,Sport
0,file_2300.txt,0.0621472,0.0421466,0.0130443,0.0038829,0.127008,0.0613026,0.196798,0.014007,0.0353658,...,0.00476286,0.00886165,0.0364149,0.0726348,0.042123,0.00388578,0.00382821,0.201259,0.0101705,0.00659258
1,file_809.txt,0.0618728,0.0424302,0.0128616,0.00384745,0.162379,0.0644784,0.195431,0.0140639,0.0337543,...,0.00473483,0.00875095,0.0326453,0.155985,0.0393143,0.00379855,0.00377858,0.0948981,0.0099209,0.00636009
2,file_1383.txt,0.0975445,0.144602,0.0133359,0.00399558,0.0758506,0.0479403,0.136187,0.0142812,0.0355864,...,0.00493601,0.00917026,0.0342179,0.0529838,0.151729,0.00399736,0.00395552,0.0972641,0.0110146,0.00689023
3,file_983.txt,0.081371,0.049697,0.0135967,0.00402333,0.103302,0.0638348,0.163699,0.0146887,0.0363712,...,0.00499925,0.00934587,0.036615,0.0638244,0.0479881,0.00404956,0.00401123,0.225464,0.0109523,0.00689013
4,file_1713.txt,0.153704,0.110646,0.0141907,0.00410832,0.0723873,0.0499341,0.18619,0.0153917,0.0376808,...,0.00506564,0.00963215,0.0720046,0.0489596,0.0497823,0.00413493,0.00411517,0.0888852,0.0111348,0.00707706


In [96]:
# test_set_preds.to_excel('test_set_preds.xlsx', index=False)

In [84]:
# pred_df = pd.read_excel("/Users/anurag/Downloads/Movie_Scripts_Sample_SubmissionL.xlsx")

In [140]:
multiclass_logloss(test_df['Labels'].values, test_set_preds[train_df.Genre.unique().tolist()].values)

[[0.06214723355175664 0.042146623610022714 0.013044332037354145 ...
  0.20125856438571393 0.010170535073625206 0.006592576535592437]
 [0.06187280816682498 0.04243018124017416 0.012861594741520026 ...
  0.0948981163189656 0.009920896808506535 0.006360092595007451]
 [0.09754452792196255 0.1446015828825774 0.013335926776471056 ...
  0.09726405512888492 0.011014628830312737 0.006890231855105332]
 ...
 [0.08640182590027343 0.11063912938749708 0.013709650461187581 ...
  0.09776645184552446 0.01073621514500798 0.006588622029522577]
 [0.11033200163939762 0.05623496173057861 0.01356362469949817 ...
  0.12585100435340937 0.010992541569449196 0.006971135126053225]
 [0.1911758586509735 0.043270688228447846 0.012024185208458112 ...
  0.13988146958682884 0.009851563910019954 0.006509688375101668]]


2.4597586095272357

In [27]:
test_set_preds

NameError: name 'test_set_preds' is not defined